In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_path = '/content/drive/MyDrive/MLOM_LAB4/Cat&Dog Dataset/Training_data'
test_path = '/content/drive/MyDrive/MLOM_LAB4/Cat&Dog Dataset/Testing_data'

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.regularizers import l2

In [ ]:
model = tf.keras.models.Sequential([
    # Note that input shape is the desired size of the image 300x300 with 3 bytes color
    #this is the first convolution
    tf.keras.layers.Conv2D(filters=16,kernel_size = 3,activation = 'relu',input_shape=(300,300,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    #this is the second convolution
    #tf.keras.layers.Conv2D(filters=16,kernel_size = 3,activation = 'relu'),
    #tf.keras.layers.MaxPooling2D(pool_size = 2),
    #this is the third convolution
    tf.keras.layers.Conv2D(filters=32,kernel_size = 3,activation = 'relu'),
    tf.keras.layers.MaxPooling2D(pool_size = 2),
    #tf.keras.layers.Dropout(0.5),

    #this is the fourth convolution
    tf.keras.layers.Conv2D(filters=64,kernel_size = 3,activation = 'relu'),
    tf.keras.layers.MaxPooling2D(pool_size = 2),

    tf.keras.layers.Dropout(0.2),
    #this is the fifth convolution
    tf.keras.layers.Conv2D(filters=64,kernel_size = 3,activation = 'relu'),
    tf.keras.layers.MaxPooling2D(pool_size = 2),
    # Flatten the result to feed into a Dnn
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512,activation='relu', kernel_regularizer=l2(0.01)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class('cat') and 1 for the other
    tf.keras.layers.Dense(1,activation='sigmoid')
    ])



In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 149, 149, 16)      0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 73, 73, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 35, 35, 64)       

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler

# Define a learning rate scheduler function
def lr_scheduler(epoch):
    return 0.001 * (0.1 ** (epoch // 10))

# Create a learning rate scheduler callback
lr_callback = LearningRateScheduler(lr_scheduler)

lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=20*1000,#20->steps_per_epoch
  decay_rate=1,
  staircase=False)

def get_optimizer():
  return tf.keras.optimizers.Adam(lr_schedule)

In [ ]:
#optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)
optimizer = get_optimizer()
model.compile(optimizer = optimizer,
              loss = 'binary_crossentropy', #binary_crossentropy because our class mode is binary(output is dog or cat)
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255) #colors are between 0-255. so we can arrange value 0-1 by dividing 255


#Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_path,
                                                    target_size=(300,300),
                                                    batch_size = 32,  #smaller batch_size slow
                                                    class_mode = 'binary') #final output is dog or cat.So it binary
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (300,300),
                                            batch_size = 32,
                                            class_mode = 'binary')



Found 633 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=20, # image_count/batch_size = 633/32 = 20
    epochs=10, #that means we go through our training dataset 20 times. we can add more epochs then dataset train more.
    validation_data=test_set,
    validation_steps=7,  # image_count/batch_size = 200/32 =
    callbacks=  [lr_callback]
)

Epoch 1/10
20/20 [==============================] - 76s 4s/step - loss: 7.3064 - accuracy: 0.5750 - val_loss: 5.2736 - val_accuracy: 0.5000 - lr: 9.9905e-04
Epoch 2/10
20/20 [==============================] - 70s 4s/step - loss: 3.7061 - accuracy: 0.6635 - val_loss: 2.8645 - val_accuracy: 0.4950 - lr: 9.9805e-04
Epoch 3/10
20/20 [==============================] - 80s 4s/step - loss: 2.1931 - accuracy: 0.6951 - val_loss: 2.1114 - val_accuracy: 0.5000 - lr: 9.9706e-04
Epoch 4/10
20/20 [==============================] - 74s 4s/step - loss: 1.4710 - accuracy: 0.7014 - val_loss: 1.3871 - val_accuracy: 0.5450 - lr: 9.9607e-04
Epoch 5/10
20/20 [==============================] - 71s 3s/step - loss: 1.1258 - accuracy: 0.7267 - val_loss: 1.2470 - val_accuracy: 0.5050 - lr: 9.9507e-04
Epoch 6/10
20/20 [==============================] - 71s 4s/step - loss: 0.9010 - accuracy: 0.7678 - val_loss: 1.2311 - val_accuracy: 0.5000 - lr: 9.9409e-04
Epoch 7/10
20/20 [==============================] - 70s 4s

In [ ]:
filename = 'best_model.h5'
model.load_weights(filename)

In [ ]:
loss, accuracy = model.evaluate(test_set)
print(f"Test accuracy: {accuracy * 100:.2f}%")


7/7 [==============================] - 3s 449ms/step - loss: 0.9825 - accuracy: 0.7000
Test accuracy: 70.00%


In [ ]:
loss, accuracy = model.evaluate(train_generator)
print(f"Test accuracy: {accuracy * 100:.2f}%")


20/20 [==============================] - 12s 586ms/step - loss: 0.3029 - accuracy: 0.9826
Test accuracy: 98.26%
